In [1]:
import zarr
import xarray as xr
import time
import dask

In [2]:
ask_workers=2

In [27]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = SLURMCluster(cores=1, memory='1.5TB',job_extra=['--partition=prepost','--account=yrf@cpu',
                                  '--ntasks=1','--cpus-per-task=1'], project='PerfTestPangeo', walltime='02:30:00')
cluster.scale(ask_workers)

c = Client(cluster)

c

/gpfswork/rech/yrf/rote001/conda/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38290 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://130.84.132.27:35074 Dashboard: http://130.84.132.27:38290/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [40]:
!squeue -u rote001
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           2123737   prepost dask-wor  rote001 PD       0:00      1 (Resources)
           2123736   prepost dask-wor  rote001  R       4:13      1 jean-zay-pp3
           2123941   prepost dask-wor  rote001  R      22:36      1 jean-zay-pp2
           2123942   prepost dask-wor  rote001  R      22:36      1 jean-zay-pp2
           2123620   prepost dask-wor  rote001  R    1:18:20      1 jean-zay-pp1
           2123621   prepost dask-wor  rote001  R    1:18:20      1 jean-zay-pp1
           2123335   prepost     srun  rote001  R    1:56:47      1 jean-zay-pp1
Workers= 1, Memory=1.50 TB


In [41]:
%time ds=xr.open_zarr('/gpfswork/rech/yrf/rote001/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

CPU times: user 145 ms, sys: 22.6 ms, total: 167 ms
Wall time: 165 ms


In [42]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 101 ms, sys: 6.5 ms, total: 108 ms
Wall time: 126 ms


In [ ]:
%time mean.load()